In [1]:
#%pip install yfinance crewai crewai-tools

In [2]:
#%pip install docker

In [3]:
import re
import json
import os
import yfinance as yf
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import CodeInterpreterTool, FileReadTool

In [4]:
from pydantic import BaseModel, Field

class QueryAnalysisOutput(BaseModel):
    """Structured output for the query analysis task."""
    symbol: str = Field(..., description="Stock ticker symbol (e.g., TSLA, AAPL).")
    timeframe: str = Field(..., description="Time period (e.g., '1d', '1mo', '1y').")
    action: str = Field(..., description="Action to be performed (e.g., 'fetch', 'plot').")

In [13]:
llm = LLM(
    model="ollama/deepseek-r1:1.5b",
    base_url="http://localhost:11434",
    # temperature=0.7
)

In [14]:
import docker
try:
    client = docker.from_env()
    client.ping()
    print("Docker daemon ping successful!")
except Exception as e:
    print(f"Error pinging Docker daemon: {e}")

Docker daemon ping successful!


In [15]:
!docker --version

Docker version 28.1.1, build 4eba377


In [16]:
# 1) Query parser agent
query_parser_agent = Agent(
    role="Stock Data Analyst",
    goal="Extract stock details and fetch required data from this user query: {query}.",
    backstory="You are a financial analyst specializing in stock market data retrieval.",
    llm=llm,
    verbose=True,
    memory=True,
)

query_parsing_task = Task(
    description="Analyze the user query and extract stock details.",
    expected_output="A dictionary with keys: 'symbol', 'timeframe', 'action'.",
    output_pydantic=QueryAnalysisOutput,
    agent=query_parser_agent,
)

# 2) Code writer agent
code_writer_agent = Agent(
    role="Senior Python Developer",
    goal="Write Python code to visualize stock data.",
    backstory="""You are a Senior Python developer specializing in stock market data visualization. 
                 You are also a Pandas, Matplotlib and yfinance library expert.
                 You are skilled at writing production-ready Python code""",
    llm=llm,
    verbose=True,
)

code_writer_task = Task(
    description="""Write Python code to visualize stock data based on the inputs from the stock analyst
                   where you would find stock symbol, timeframe and action.""",
    expected_output="A clean and executable Python script file (.py) for stock visualization.",
    agent=code_writer_agent,
)

# 3) Code interpreter agent (uses code interpreter tool from crewai)
code_interpreter_tool = CodeInterpreterTool()

code_execution_agent = Agent(
    role="Senior Code Execution Expert",
    goal="Review and execute the generated Python code by code writer agent to visualize stock data.",
    backstory="You are a code execution expert. You are skilled at executing Python code.",
    # tools=[code_interpreter_tool],
    allow_code_execution=True,   # This automatically adds the CodeInterpreterTool
    llm=llm,
    verbose=True,
)

code_execution_task = Task(
    description="""Review and execute the generated Python code by code writer agent to visualize stock data.""",
    expected_output="A clean and executable Python script file (.py) for stock visualization.",
    agent=code_execution_agent,
)

In [17]:
### --- CREW SETUP --- ###

crew = Crew(
    agents=[query_parser_agent, code_writer_agent, code_execution_agent],
    tasks=[query_parsing_task, code_writer_task, code_execution_task],
    process=Process.sequential
)

# Run the crew with an example query
result = crew.kickoff(inputs={"query": "Plot YTD stock gain of Tesla"})

# Agent: Stock Data Analyst
## Task: Analyze the user query and extract stock details.


# Agent: Stock Data Analyst
## Final Answer: 
<think>
Okay, so I need to figure out what information is being requested here. The user provided a query: "Plot YTD stock gain of Tesla." And my task is to extract stock details and fetch required data from this user query.

First, let me understand the components needed in the dictionary. The keys are 'symbol', 'timeframe', and 'action'. So I need to identify which parts correspond to each key.

Looking at the query: "Plot YTD stock gain of Tesla." The main elements here are the company name (Tesla), the timeframe ("YTD"), and the action is about plotting or visualizing the stock gain.

Now, for the 'symbol', it's straightforward. The stock symbol is the abbreviation that represents a specific stock. For Tesla, I recall the symbol is "TSLA".

For the timeframe, the user specified YTD, which stands for Year-to-Date. So the timeframe key should be "YTD"